# Sample training script with naive competitive self-play.

### Switch directory in Google drive so as to import CDA env.


In [134]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

# %cd "/root/ray_results/"
# !ls -l
# #!rm -rf PPO_continuousDoubleAuction-v0_*
# !ls -l
# !pwd

# %cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
# !ls -l

# #!pip install -r requirements.txt

# #!pip install tensorflow==2.2.0
# #!pip install ray[rllib]==0.8.5

# #!pip show tensorflow
# #!pip show ray

# #!pip freeze > requirements.txt

In [135]:
# !pip install sortedcontainers

In [136]:
# !!pip install scikit-learn

In [137]:
# !pip install tabulate

In [138]:
# !pip install -U ipywidgets

In [139]:
# !pip install matplotlib

### Imports

In [140]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse



# import gym
import gymnasium as gym



import random
import numpy as np

from collections import defaultdict
from typing import Dict

import ray
from ray import tune
from ray.rllib.utils import try_import_tf
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.policy import Policy


# from ray.rllib.agents.ppo import ppo
# from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.algorithms.ppo import PPOConfig



from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.sample_batch import SampleBatch



# from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
# from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.algorithms.callbacks import DefaultCallbacks



from ray.tune.logger import pretty_print

import sys
if "../" not in sys.path:
    sys.path.append("../")




from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv



# from gym_continuousDoubleAuction.train.model.model_handler import CustomModel_1
from gym_continuousDoubleAuction.train.model.model_handler import CustomLSTMRLModule



from gym_continuousDoubleAuction.train.policy.policy_handler import make_RandomPolicy, gen_policy, set_agents_policies, create_train_policy_list
from gym_continuousDoubleAuction.train.weight.weight_handler import get_trained_policies_name, get_max_reward_ind, cp_weight
from gym_continuousDoubleAuction.train.storage.store_handler import storage
from gym_continuousDoubleAuction.train.callbk.callbk_handler import store_eps_hist_data
from gym_continuousDoubleAuction.train.logger.log_handler import create_dir, log_g_store, load_g_store
from gym_continuousDoubleAuction.train.plotter.plot_handler import plot_storage, plot_LOB_subplot, plot_sum_ord_imb, plot_mid_prices
from gym_continuousDoubleAuction.train.helper.helper import ord_imb, sum_ord_imb, mid_price

# tf = try_import_tf()
print(f'Imports all OK.')

Imports all OK.


### Global


In [141]:
# CDA_env args
num_agents = 9
num_trained_agent = 3 # 
num_policies = num_agents # Each agent is using a separate policy
num_of_traders = num_agents
tape_display_length = 10 
tick_size = 1
init_cash = 1000000
max_step = 4096 # per episode, -1 in arg. (~7.2s/1000steps/iter)
is_render = False 

# RLlib config 
train_policy_list = create_train_policy_list(num_trained_agent, "policy_")
#num_cpus = 0.25                                
num_gpus = 0.75 #0                       
num_cpus_per_worker = 0.25                                
num_gpus_per_worker = 0
num_workers = 2
num_envs_per_worker = 4
batch_mode = "complete_episodes" 
rollout_fragment_length = 128
train_batch_size = max_step
sgd_minibatch_size = 256
num_iters = 2

# log_base_dir = "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/results/"
log_base_dir = "results/"
log_dir = log_base_dir + "ray_results/"

# Chkpt & restore
local_dir = log_base_dir + "chkpt/"
chkpt_freq = 10
chkpt = 320
restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)
is_restore = True # True / False

# log & load 
log_g_store_dir = log_base_dir + "log_g_store/"
create_dir(log_base_dir)    
create_dir(log_g_store_dir)    

# get obs & act spaces from dummy CDA env
single_CDA_env = continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step, is_render)
obs_space = single_CDA_env.observation_space
act_space = single_CDA_env.action_space

# register CDA env with RLlib 
register_env("continuousDoubleAuction-v0", lambda _: continuousDoubleAuctionEnv(num_of_traders, 
                                                                                init_cash, 
                                                                                tick_size, 
                                                                                tape_display_length,
                                                                                max_step-1, 
                                                                                is_render))

# register custom model (neural network)
# ModelCatalog.register_custom_model("model_disc", CustomModel_1) 
ModelCatalog.register_custom_model("model_disc", CustomLSTMRLModule) 

ray.shutdown()
# start ray
# ray.init(ignore_reinit_error=True, log_to_driver=True, webui_host='127.0.0.1', num_cpus=2) 
# ray.init(ignore_reinit_error=True, log_to_driver=True, dashboard_host="127.0.0.1", num_cpus=2)
ray.init(
    ignore_reinit_error=True,
    log_to_driver=True,
    num_cpus=2,
    dashboard_host="127.0.0.1",  # replaces webui_host
    dashboard_port=8265,          # default port; replaces webui_port
    # include_dashboard=True,        # default True
    include_dashboard=False,        # default True

)

# Global storage, a ray actor that run on it's own process & it needs to be declared after ray.init().
# g_store = storage.options(name="g_store", detached=True).remote(num_agents)
g_store = storage.options(name="g_store", lifetime="detached").remote(num_agents)

train_policy_list =  ['policy_0', 'policy_1', 'policy_2']
Folder creation failed or folder already exists: results/
Folder creation failed or folder already exists: results/log_g_store/


2025-07-12 00:03:32,817	WARNING services.py:2152 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.82gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-07-12 00:03:33,867	INFO worker.py:1917 -- Started a local Ray instance.


(raylet) It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="888efc26-28ed-45be-9b76-448f3f238800", ...)


### Policies

In [142]:
# Dictionary of policies
policies = {"policy_{}".format(i): gen_policy(i, obs_space, act_space) for i in range(num_policies)}
set_agents_policies(policies, obs_space, act_space, num_agents, num_trained_agent)
policy_ids = list(policies.keys())

def policy_mapper(agent_id):
    """
    Required in RLlib config.
    """
    for i in range(num_agents):
        if agent_id == i:            
            return "policy_{}".format(i)                

policies: {'policy_0': (None, Box(-inf, inf, (4, 10), float32), Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), {}), 'policy_1': (None, Box(-inf, inf, (4, 10), float32), Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), {}), 'policy_2': (None, Box(-inf, inf, (4, 10), float32), Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), {}), 'policy_3': (<class 'gym_continuousDoubleAuction.train.policy.policy_handler.make_RandomPolicy.<locals>.RandomPolicy'>, Box(-inf, inf, (4, 10), float32), Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), {}), 'policy_4': (<class 'gym_continuousDoubleAuction.train.policy.policy_handler.make_RandomPolicy.<locals>.RandomPolicy'>, Box(-inf, inf, (4, 10), float32), Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0

### Call back

In [143]:
class MyCallbacks(DefaultCallbacks):
    # def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
    #                      policies: Dict[str, Policy],
    #                      episode: MultiAgentEpisode, **kwargs):
    def on_episode_start(self, *, worker, base_env, policies, episode, **kwargs):
        print(f"Episode {episode.episode_id} started.")
        """
        info["episode"] is a MultiAgentEpisode object.

        user_data dicts at 100000 items max, will auto replace old with new item at 1st index.
        hist_data dicts at 100 items max, will auto replace old with new item at 1st index.
        """
        #print("on_episode_start {}, _agent_to_policy {}".format(episode.episode_id, episode._agent_to_policy))

        prefix = "agt_"
        for i in range(num_agents):
            episode.user_data[prefix + str(i) + "_obs"] = []
            episode.user_data[prefix + str(i) + "_act"] = []
            episode.user_data[prefix + str(i) + "_reward"] = []
            episode.user_data[prefix + str(i) + "_NAV"] = []
            episode.user_data[prefix + str(i) + "_num_trades"] = []

            episode.hist_data[prefix + str(i) + "_reward"] = []
            episode.hist_data[prefix + str(i) + "_NAV"] = []
            episode.hist_data[prefix + str(i) + "_num_trades"] = []

    # def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
    #                     episode: MultiAgentEpisode, **kwargs):
    def on_episode_step(self, *, worker, base_env, episode, **kwargs):
        # Example: track something each step
        episode.user_data["my_metric"].append(1)        
        """
        pole_angle = abs(episode.last_observation_for()[2])
        raw_angle = abs(episode.last_raw_obs_for()[2])
        assert pole_angle == raw_angle
        episode.user_data["pole_angles"].append(pole_angle)
        """

        prefix = "agt_"
        for i in range(num_agents):
            obs = episode.last_raw_obs_for(i)
            #obs = episode.last_observation_for(i)
            act = episode.last_action_for(i)
            reward = episode.last_info_for(i).get("reward")
            NAV = episode.last_info_for(i).get("NAV")
            NAV = None if NAV is None else float(NAV)
            num_trades = episode.last_info_for(i).get("num_trades")
        
            if reward is None:      # goto next agent.
                continue

            episode.user_data[prefix + str(i) + "_obs"].append(obs)    
            episode.user_data[prefix + str(i) + "_act"].append(act)    
            episode.user_data[prefix + str(i) + "_reward"].append(reward)    
            episode.user_data[prefix + str(i) + "_NAV"].append(NAV)    
            episode.user_data[prefix + str(i) + "_num_trades"].append(num_trades)          

    # def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
    #                    policies: Dict[str, Policy], episode: MultiAgentEpisode,
    #                    **kwargs):
    def on_episode_end(self, *, worker, base_env, policies, episode, **kwargs):
        # Example: summarize at end
        mean_metric = sum(episode.user_data["my_metric"]) / len(episode.user_data["my_metric"])
        print(f"Episode {episode.episode_id} ended. Mean my_metric: {mean_metric}")        
        #print("on_episode_end {}, episode.agent_rewards {}".format(episode.episode_id, episode.agent_rewards))
        """
        arg: {"env": .., "episode": ...}
        """

        g_store = ray.util.get_actor("g_store")
        prefix = "agt_"
        for agt_id in range(num_agents):
            obs_key = prefix + str(agt_id) + "_obs"
            act_key = prefix + str(agt_id) + "_act"
            reward_key = prefix + str(agt_id) + "_reward"
            NAV_key = prefix + str(agt_id) + "_NAV"
            num_trades_key = prefix + str(agt_id) + "_num_trades"      

            # store into episode.hist_data
            store_eps_hist_data(episode, reward_key)
            store_eps_hist_data(episode, NAV_key)
            store_eps_hist_data(episode, num_trades_key)

            # store step data
            obs = episode.user_data[obs_key]
            act = episode.user_data[act_key]
            reward = episode.user_data[reward_key]
            NAV = episode.user_data[NAV_key]
            num_trades = episode.user_data[num_trades_key]
            ray.get(g_store.store_agt_step.remote(agt_id, obs, act, reward, NAV, num_trades))       
        
            # Store episode data.   
            eps_reward = np.sum(reward)
            eps_NAV = np.sum(NAV)
            eps_num_trades = np.sum(num_trades)
            ray.get(g_store.store_agt_eps.remote(agt_id, eps_reward, eps_NAV, eps_num_trades))

        ray.get(g_store.inc_eps_counter.remote())  

    # def on_sample_end(self, worker: RolloutWorker, samples: SampleBatch,
    #                   **kwargs):
    #     """
    #     arg: {"samples": .., "worker": ...}

    #     Notes:
    #         https://github.com/ray-project/ray/blob/master/rllib/policy/sample_batch.py        
    #     """
    #     #print("on_sample_end returned sample batch of size {}".format(samples.count))

    #     """
    #     MultiAgentBatch_obj = info["samples"]
    #     MultiAgentBatch_policy_batches = MultiAgentBatch_obj.policy_batches
    #     MultiAgentBatch_count = MultiAgentBatch_obj.count

    #     access_sample_batches(MultiAgentBatch_policy_batches)
    #     print("info['samples'].policy_batches = {}".format(info["samples"].policy_batches))
    #     print("info['worker'] = {}".format(info["worker"])) # RolloutWorker object
    #     """

    def on_train_result(self, trainer, result: dict, **kwargs):
        """
        info["trainer"] is the trainer object.

        info["result"] contains a bunch of info such as episodic rewards for 
        each policy in info["result"][hist_stats] dictionary.
        """
        #print("trainer.train() result: {} -> {} episodes".format(trainer, result["episodes_this_iter"]))
        # you can mutate the result dict to add new fields to return
        result["callback_ok"] = True
        #print("on_train_result result", result)
    
        train_policies_name = get_trained_policies_name(policies, num_trained_agent)    
        max_reward_ind = get_max_reward_ind(result, train_policies_name)
        max_reward_policy_name = train_policies_name[max_reward_ind]
        cp_weight(trainer, train_policies_name, max_reward_policy_name)    

        g_store = ray.util.get_actor("g_store")      
        prefix = "policy_policy_"
        suffix = "_reward"
        hist_stats = result["hist_stats"]
        eps_this_iter = result["episodes_this_iter"]
        for agt_id in range(num_agents):
            key = prefix + str(agt_id) + suffix
            for i in range(eps_this_iter):
                ray.get(g_store.store_agt_train.remote(agt_id, hist_stats[key][i]))

        #print("on_train_result info['result'] {}".format(info["result"]))     

### RLlib config

In [144]:
# def get_config():
#     config = ppo.DEFAULT_CONFIG.copy()
#     config["multiagent"] = {"policies_to_train": train_policy_list,
#                             "policies": policies,
#                             "policy_mapping_fn": policy_mapper,
#                            }    
#     #config["num_cpus"] = num_cpus     # trainer, applicable only when using tune.
#     config["num_gpus"] = num_gpus     # trainer
#     config["num_cpus_per_worker"] = num_cpus_per_worker                                
#     config["num_gpus_per_worker"] = num_gpus_per_worker                      
#     config["num_workers"] = num_workers
#     config["num_envs_per_worker"] = num_envs_per_worker  
#     config["batch_mode"] = batch_mode       # "complete_episodes" / "truncate_episodes"
#     config["train_batch_size"] = train_batch_size # Training batch size, if applicable. Should be >= rollout_fragment_length.
#                                                   # Samples batches will be concatenated together to a batch of this size,
#                                                   # which is then passed to SGD.
#     config["rollout_fragment_length"] = rollout_fragment_length # replaced "sample_batch_size",
#     config["sgd_minibatch_size"] = sgd_minibatch_size 
#     config["log_level"] = "WARN" # WARN/INFO/DEBUG 
#     config["callbacks"] = MyCallbacks
#     config["output"] = log_dir

#     return config

In [145]:
def get_config():
    config = (
        PPOConfig()
        .framework("torch")  # or "tf"
        .env_runners(
            num_env_runners=num_workers,              # renamed param here
            num_envs_per_env_runner=num_envs_per_worker,  # renamed param here
            rollout_fragment_length=rollout_fragment_length,
            batch_mode=batch_mode,
        )
        .training(
            train_batch_size=train_batch_size,
            sgd_minibatch_size=sgd_minibatch_size,
        )
        .resources(
            num_gpus=num_gpus,
            num_cpus_per_worker=num_cpus_per_worker,
            num_gpus_per_worker=num_gpus_per_worker,
        )
        .multi_agent(
            policies=policies,
            policy_mapping_fn=policy_mapper,
            policies_to_train=train_policy_list,
        )
        .callbacks(MyCallbacks)
        .debugging(log_level="WARN")
        .output(log_dir)
    )
    return config

### Train


In [146]:
def go_train(config):    
    trainer = ppo.PPOTrainer(config=config, env="continuousDoubleAuction-v0")
    
    if is_restore == True:
        trainer.restore(restore_path) 

    g_store = ray.util.get_actor("g_store")          
    result = None
    for i in range(num_iters):
        result = trainer.train()       
        print(pretty_print(result)) # includes result["custom_metrics"]
        print("training loop = {} of {}".format(i + 1, num_iters))            
        print("eps sampled so far {}".format(ray.get(g_store.get_eps_counter.remote())))

        if i % chkpt_freq == 0:
            checkpoint = trainer.save(local_dir)
            print("checkpoint saved at", checkpoint)
    
    checkpoint = trainer.save(local_dir)
    print("checkpoint saved at", checkpoint)

    print("result['experiment_id']", result["experiment_id"])
    
    return result["experiment_id"]
    
# run everything
experiment_id = go_train(get_config())            

TypeError: AlgorithmConfig.training() got an unexpected keyword argument 'sgd_minibatch_size'

### Plot all steps.

Agt_0, 1, 2 are trained agents (with PPO) while the rest are random agents.

In [ ]:
# plot_storage(num_agents, init_cash)

In [ ]:
# plot_storage(num_agents, init_cash, "step", "NAV")

In [ ]:
# plot_storage(num_agents, init_cash, "step", "num_trades")

### Log/load last episode

In [ ]:
# log_g_store(log_g_store_dir, num_agents, experiment_id)
# load_g_store(log_g_store_dir, num_agents, experiment_id)

### Plot steps from last episode.

In [ ]:
# plot_storage(num_agents, init_cash)

In [ ]:
# plot_storage(num_agents, init_cash, "step", "NAV")

In [ ]:
# plot_storage(num_agents, init_cash, "step", "num_trades")

### LOB from last episode.

In [ ]:
# g_store = ray.util.get_actor("g_store")          
# #store = ray.get(g_rere.get_storage.remote())

# depth = 10
# bid_size, bid_price, ask_size, ask_price = ray.get(g_store.get_obs_from_agt.remote(0, depth))

### LOB order imbalance

In [ ]:
# ord_imb_store = ord_imb(bid_size, ask_size)
# plot_LOB_subplot(ord_imb_store, depth, '_ord_imb') 

### LOB sum of order imbalance

In [ ]:
# ord_imb_store = np.asarray(ord_imb_store)
# sum_ord_imb_store = sum_ord_imb(ord_imb_store)
# plot_sum_ord_imb(sum_ord_imb_store, "sum_ord_imb")

### LOB mid price (subplot)

In [ ]:
# mid_price_store = mid_price(bid_price, ask_price)
# plot_LOB_subplot(mid_price_store, depth, '_mid_price')    

### LOB mid prices 

In [ ]:
# plot_mid_prices(mid_price_store,)

### LOB bid size

In [ ]:
# plot_LOB_subplot(bid_size, depth, '_bid_size')    

### LOB ask size

In [ ]:
# plot_LOB_subplot(ask_size, depth, '_ask_size')    

### LOB bid price

In [ ]:
# plot_LOB_subplot(bid_price, depth, '_bid_price')    

### LOB ask price

In [ ]:
# plot_LOB_subplot(ask_price, depth, '_ask_price')    